In [23]:
import pandas as pd
import numpy as np
import gzip
import os
import datetime
import pandas as pd
import calendar
import numpy as np
import glob
import sys
import gc
import re
#import quantstats as qs
import time

import matplotlib.pyplot as plt

import nest_asyncio # install this package to avoid running in loop

import urllib
import pandas_ta as ta

import warnings

import requests

import hmac
import hashlib

import aiohttp
import asyncio

import logging
from datetime import timezone

import nest_asyncio

# Filter depreciation warnings from pandas regarding the append method
warnings.filterwarnings('ignore', category=FutureWarning, message=".*append.*")
pd.options.mode.chained_assignment = None

# Set pandas to display all digits for floating-point numbers
pd.options.display.float_format = '{:.8f}'.format

import os
from dotenv import load_dotenv

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Ignore only FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

# Ignore only RuntimeWarning
warnings.filterwarnings('ignore', category=RuntimeWarning)

import importlib


from Orats_options_fetch_functions import *

from faster_numba_strategy_functions import *

import strategy_functions

importlib.reload(strategy_functions)
from strategy_functions import *

# import Common_functions

# importlib.reload(strategy_functions)
# importlib.reload(Common_functions)

# from Common_functions import *
# coin_desk_api_key =  "e43c1fd991e660b9bf959645f0800bb7e76fb4a3537ab773cec62b2fad31af2d"
from datetime import time
from pathlib import Path
# Get the user's home directory dynamically
home_dir = r"C:\Data\Options_logs"
#====================================

strategy_name = "Put_buy_0DTE"

# Create a folder for the strategy_name if it doesn't exist in home directory

strategy_folder = os.path.join(home_dir, strategy_name)
if not os.path.exists(strategy_folder):
    os.makedirs(strategy_folder)

# Create the full path to the output file. this file contains all the trades

output_all_trades_file = os.path.join(strategy_folder, f"All_trades_{strategy_name}.csv")
trade_html_file = os.path.join(strategy_folder, f"Trade_stats_{strategy_name}.html")

#=== location to store the trade dataframe for debug

base_folder_df = "strategy_dataframe"
df_folder = os.path.join(strategy_folder, base_folder_df)
if not os.path.exists(df_folder):
    os.makedirs(df_folder)
    print("\n Created strategy dataframe folder : " ,df_folder )

#----- symbol_wise_trades

trades_folder = "sym_wise_trades"
trades_strategy_folder = os.path.join(strategy_folder, trades_folder)

if not os.path.exists(trades_strategy_folder):
    print("\n Making folder for symbol wise trades of strategy" , trades_strategy_folder)
    os.makedirs(trades_strategy_folder)

logger = setup_logger("Straddle_0DTE", strategy_folder, log_file="strategy.log" , to_console=False)

In [24]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

# Connect to an in-memory database
con = duckdb.connect()

# --- Your setup code (which is correct) ---
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute("""
SET s3_endpoint='122.176.133.73:9000';
SET s3_use_ssl=false;
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
""")
con.execute("SET s3_url_style='path';") 
con.execute("PRAGMA threads=8;")
con.execute("PRAGMA enable_object_cache;")

# --- THE MISSING STEP: Querying a file from S3 ---
# Replace with your actual S3 bucket and file path
# This can be a .parquet, .csv, or .json file
s3_file_path = 's3://duckdata/ORATS/Options/**/*.parquet'

try:
    # Execute a query on the S3 file and fetch the result as a Pandas DataFrame
    print(f"Querying data from: {s3_file_path}")
    df = con.execute(f"""
        SELECT *
        FROM '{s3_file_path}'
        LIMIT 2;
    """).df()

    print("\n✅ Query successful! First 5 rows of data:")
    print(df.head())

except Exception as e:
    print(f"\n❌ An error occurred during the query: {e}")

# finally:
#     # It's good practice to close the connection
#     con.close()

Querying data from: s3://duckdata/ORATS/Options/**/*.parquet

✅ Query successful! First 5 rows of data:
      expiry       strike  dte  optionType                  ts         low  \
0 2024-01-26 421.00000000   25          -1 2024-01-02 17:13:00 17.18000000   
1 2024-01-12 414.00000000   11          -1 2024-01-02 17:13:00 10.41500000   

         high        open       close     volume  ...      askSize  \
0 17.18000000 17.18000000 17.18000000 0.00000000  ...  50.00000000   
1 10.41500000 10.41500000 10.41500000 0.00000000  ... 151.00000000   

     bidPrice    askPrice      bidIv      askIv         iv          oi  \
0 17.12000000 17.24000000 0.13785700 0.14511500 0.14148600 15.00000000   
1 10.34000000 10.49000000 0.14267100 0.15199600 0.14733300 54.00000000   

    stockPrice        day ticker  
0 404.00000000 2024-01-02    QQQ  
1 404.00000000 2024-01-02    QQQ  

[2 rows x 21 columns]


In [25]:

# Example 1: 0DTE At-the-Money LONG STRADDLE
strategy_config = {
    'Leg1': {'action': 'SELL', 'option_type': -1, 'expiry_offset': 0, 'strike_offset': 1}  # Sell ATM Call
    }   # Buy 1DTE call



In [35]:
com_df['SPY_close'].iloc[150:200]

option_ts
2024-01-04 11:47:00   470.14000000
2024-01-04 11:53:00   469.64000000
2024-01-04 12:04:00   469.47000000
2024-01-04 12:36:00   469.14000000
2024-01-04 12:50:00   468.95000000
2024-01-04 12:53:00   468.98000000
2024-01-04 12:57:00   469.00000000
2024-01-04 13:00:00   469.12000000
2024-01-04 13:11:00   469.25000000
2024-01-05 09:36:00   468.15000000
2024-01-05 09:38:00   468.14000000
2024-01-05 09:39:00   468.20000000
2024-01-05 09:40:00   467.94000000
2024-01-05 09:41:00   467.67000000
2024-01-05 09:42:00   467.92000000
2024-01-05 09:43:00   467.99000000
2024-01-05 09:44:00   468.06000000
2024-01-05 09:45:00   468.01000000
2024-01-05 09:46:00   468.37000000
2024-01-05 09:47:00   468.38000000
2024-01-05 09:49:00   468.32000000
2024-01-05 09:51:00   468.42000000
2024-01-05 09:52:00   468.47000000
2024-01-05 09:54:00   468.50000000
2024-01-05 09:55:00   468.40000000
2024-01-05 09:57:00   468.19000000
2024-01-05 09:58:00   468.12000000
2024-01-05 09:59:00   468.11000000
2024-01-05

In [26]:


ifol = r"C:\Data\Aggregate_Data\Options\USA\SPY_1min_2024_2025_ORATS_close.parquet"
#== read the RELIANCE data

com_df2 = pd.read_parquet(ifol)

com_df2 = com_df2.drop_duplicates(keep='first')

com_df2.dropna(how='all', inplace=True)

vix_df = pd.read_parquet(r"C:\Data\Aggregate_Data\VIX_1_min_2019_2025.parquet")
com_df2 = com_df2.join(vix_df)


final_symbols = com_df2.filter(like='close').columns.str.replace('_close', '')
print("Total symbols found after all filtering: ", len(final_symbols))

# Calculate the number of rows in the DataFrame

com_df2[[f"{symbol}_intraday_low" for symbol in final_symbols]] =com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: intraday_low(com_df2, low_col=x.name,agg_func='min'))

com_df2[[f"{symbol}_intraday_high" for symbol in final_symbols]] =com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: intraday_high(com_df2, high_col=x.name,agg_func='max'))

com_df2[[f"{symbol}_todayo" for symbol in final_symbols]] = com_df2[[f'{symbol}_close' for symbol in final_symbols]].resample('D').transform('first')

# com_df2[[f"{symbol}_daily_ROC" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change(periods=50)

# com_df2[[f"{symbol}_ROC" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change()

# com_df2[[f"{symbol}_ROC_4" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change(periods=4)

com_df2[[f"{symbol}_prevdayl" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_low_numba(com_df2, n=1, column=x.name))

com_df2[[f"{symbol}_prevdayh" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=1, column=x.name))

com_df2[[f"{symbol}_secdayh" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=2, column=x.name))


# This single call does it for the 20-day MA
ma_20d_df = calculate_daily_ma_vectorized(df=com_df2, symbols=final_symbols, periods=3 , agg_func='last' , col_suffix='close')

    # Join the new features back to the main DataFrame
com_df2 = pd.concat([com_df2, ma_20d_df] , axis=1 , join='outer')


com_df2[[f"{symbol}_ma" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(2).mean()

com_df2[[f"{symbol}_ROC_30" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma" for symbol in final_symbols]].pct_change(periods=30)


#com_df2[[f"{symbol}_ma_3d_low" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma" for symbol in final_symbols]].values / com_df2[[f"{symbol}_3d_MA" for symbol in final_symbols]].values

com_df2[[f"{symbol}_prevdayc" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: calculate_prev_day_close(com_df2, close_col=x.name, n_days=1))

com_df2[[f"{symbol}_secdayc" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: calculate_prev_day_close(com_df2, close_col=x.name, n_days=2))


# #=== hourly volume
# com_df2[[f"{symbol}_vol_ma" for symbol in final_symbols]] = com_df2[[f"{symbol}_volume" for symbol in final_symbols]].rolling(4).mean()

# #==avg 3 day volume

# com_df2[[f"{symbol}_3d_vol_MA" for symbol in final_symbols]] = com_df2[[f"{symbol}_volume" for symbol in final_symbols]].apply(lambda x: daily_moving_average(com_df2, timeframe='1D', column=x.name, periods=3, agg_func='last'))

com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(5).mean()


com_df2[[f"{symbol}_3dlow" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]].apply(lambda x: get_x_day_low_numba(com_df2, n=3, column=x.name))

com_df2[[f"{symbol}_3dhigh" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=3, column=x.name))



# 1. Get the lists of column names
high_3d_cols = [f"{symbol}_3dhigh" for symbol in final_symbols]
low_3d_cols = [f"{symbol}_3dlow" for symbol in final_symbols]

# 2. Perform the calculation on NumPy arrays
range_values = (com_df2[high_3d_cols].values / com_df2[low_3d_cols].values) - 1

# 3. Create a new DataFrame from the results
# This is the key step to avoid the error.
range_cols = [f"{symbol}_3d_rng" for symbol in final_symbols]
range_df = pd.DataFrame(range_values, index=com_df2.index, columns=range_cols)

# 4. Join the new DataFrame back to the main one
com_df2 = com_df2.join(range_df)


# 1. Define the column groups
close_cols = [f"{symbol}_close" for symbol in final_symbols]
low_3d_cols = [f"{symbol}_3dlow" for symbol in final_symbols]
high_3d_cols = [f"{symbol}_3dhigh" for symbol in final_symbols]

# 2. Perform the calculation in a more chained and memory-conscious way
# Using .to_numpy(dtype=np.float32) can cut memory usage by 50%
numerator = com_df2[close_cols].to_numpy(dtype=np.float32) - com_df2[low_3d_cols].to_numpy(dtype=np.float32)
denominator = com_df2[high_3d_cols].to_numpy(dtype=np.float32) - com_df2[low_3d_cols].to_numpy(dtype=np.float32)

# np.divide handles division by zero safely
percentile_values = np.divide(numerator, denominator, where=denominator != 0)

# 3. Create the new feature DataFrame
pctl_cols = [f"{symbol}_3d_Pctl" for symbol in final_symbols]
percentile_df = pd.DataFrame(percentile_values, index=com_df2.index, columns=pctl_cols)

# 4. Clip values and join back (same as your code)
percentile_df = percentile_df.clip(0, 1)
com_df2 = com_df2.join(percentile_df)

com_df2['decent'] = np.where((com_df2['SPY_todayo'] < com_df2['SPY_prevdayc'] * 1.003) & (com_df2['SPY_todayo'] > com_df2['SPY_prevdayc'] * 0.98), 1, 0)
com_df = com_df2[com_df2['decent']==1]
com_df = com_df[com_df.index.year>=2024]

Total symbols found after all filtering:  2
Calculating 3-day MA for 2 symbols...


In [37]:
com_df['SPY_close']

option_ts
2024-01-03 09:30:00   470.46000000
2024-01-03 09:31:00   470.37000000
2024-01-03 09:32:00   470.47000000
2024-01-03 09:33:00   470.23000000
2024-01-03 09:34:00   470.15000000
                          ...     
2024-12-23 13:16:00   593.32000000
2024-12-23 13:22:00   593.22000000
2024-12-23 13:23:00   592.97000000
2024-12-23 13:24:00   593.05000000
2024-12-23 13:25:00   592.96000000
Name: SPY_close, Length: 7217, dtype: float64

In [ ]:
import pandas as pd
import pandas_market_calendars as mcal

def fetch_option_data_for_n_days(con, ticker, end_date_str, n_days, strike, expiry_str, option_type):
    """
    Fetches a range of minute-level data for a single option contract
    for a specified number of business days ending on a given date.

    Args:
        con: The DuckDB connection object.
        ticker (str): The stock ticker (e.g., 'SPY').
        end_date_str (str): The last day to query, in 'YYYY-MM-DD' format.
        n_days (int): The number of business days of data to fetch.
        strike (float): The specific strike price.
        expiry_str (str): The expiry date to filter for, in 'YYYY-MM-DD' format.
        option_type (int): The type of option to fetch (1 for Calls, 0 for Puts).

    Returns:
        pd.DataFrame: A DataFrame containing the minute-by-minute data.
    """
    option_name = "Call" if option_type == 1 else "Put"
    #print(f"Fetching last {n_days} b-days of data ending on {end_date_str} for {ticker} {strike} {option_name} expiring {expiry_str}...")
    
    try:
        # --- Business Day Calculation Logic ---
        # 1. Get the market calendar
        nyse = mcal.get_calendar('NYSE')
        
        # 2. To be safe, find a start date far enough in the past
        #    (n * 2 is a safe buffer for weekends/holidays)
        start_buffer = pd.to_datetime(end_date_str) - pd.Timedelta(days=n_days * 2)
        
        # 3. Generate a schedule of valid trading days
        schedule = nyse.schedule(start_date=start_buffer, end_date=end_date_str)
        
        # 4. Get the last 'n_days' from the schedule and format them
        business_day_list = [d.strftime('%Y-%m-%d') for d in schedule.index[-n_days:]]
        
        # --- Query Logic (remains the same) ---
        path_list = [
            f"'s3://duckdata/ORATS/Options/ticker={ticker}/day={d}/*.parquet'"
            for d in business_day_list
        ]
        
        query = f"""
            SELECT ts, strike, expiry, dte, optionType, volume, oi, close,
                bidPrice, askPrice, iv, stockPrice, ticker
            FROM read_parquet([{",".join(path_list)}])
            WHERE 
                CAST(strike AS FLOAT) = {strike} AND
                expiry = '{expiry_str}' AND
                optionType = {option_type}
            ORDER BY ts;
        """
        
        multi_day_df = con.execute(query).df()
        
        if multi_day_df.empty:
            print("INFO: No data found for this contract in the specified date range.")
        
        multi_day_df['ts'] = pd.to_datetime(multi_day_df['ts'].dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None))
        multi_day_df.set_index('ts' , inplace=True)
        return multi_day_df

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# ==============================================================================
# --- Example Usage ---
# ==============================================================================
# Assume 'con' is your active DuckDB connection

# Get data for the 5 business days ending on September 12, 2025
multi_day_df = fetch_option_data_for_n_days(
    con=con,
    ticker='SPY',
    end_date_str='2024-04-12', # The last day of the range
    n_days=3,                  # How many business days to go back
    strike=510.0,
    expiry_str='2024-04-12',
    option_type=1              # 1 for Call
)

if not multi_day_df.empty:
    print("\n--- Found Data for Date Range ---")
    print(multi_day_df)

In [ ]:
from typing import List

def fetch_bulk_option_data(con, ticker, end_date_str, n_days, strikes: List[float], expiries: List[str], option_type):
    """
    Fetches a range of minute-level data for MULTIPLE option contracts
    in a single, efficient query.
    """
    # 1. Get the list of business days to query (same as before)
    nyse = mcal.get_calendar('NYSE')
    start_buffer = pd.to_datetime(end_date_str) - pd.Timedelta(days=n_days)
    schedule = nyse.schedule(start_date=start_buffer, end_date=end_date_str)
    business_day_list = [d.strftime('%Y-%m-%d') for d in schedule.index[-n_days:]]
    
    # 2. Build the list of S3 paths (same as before)
    path_list = [
        f"'s3://duckdata/ORATS/Options/ticker={ticker}/day={d}/*.parquet'"
        for d in business_day_list ]
    
    # ▼▼▼ NEW QUERY LOGIC ▼▼▼
    # 3. Format the lists for the SQL 'IN' clause
    strikes_str = ",".join(map(str, strikes)) # For numbers: 470.0,471.0,472.0
    expiries_str = ",".join([f"'{e}'" for e in expiries]) # For strings: '2024-01-05','2024-01-08'
    
    query = f"""
        SELECT ts, strike, expiry, close, bidPrice, askPrice, volume, oi, dte, optionType , iv
        FROM read_parquet([{",".join(path_list)}])
        WHERE 
            CAST(strike AS FLOAT) IN ({strikes_str}) AND
            expiry IN ({expiries_str}) AND
            optionType = {option_type}
        ORDER BY ts;
    """
    
    try:
        bulk_df = con.execute(query).df()
        
        if bulk_df.empty:
            return pd.DataFrame()
            
        # Perform timezone conversion and set index (same as before)
        bulk_df['ts'] = pd.to_datetime(bulk_df['ts'].dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None))
        bulk_df.set_index('ts' , inplace=True)
        return bulk_df
        
    except Exception as e:
        print(f"An error occurred during bulk fetch: {e}")
        return pd.DataFrame()

In [38]:

strike_interval = 1
lookback_days = 1
option_multiplier = 100
from collections import  defaultdict

# ==============================================================================
# SCRIPT STARTS HERE
# ==============================================================================

trade_log = []
nyse = mcal.get_calendar('NYSE') 
unique_days = com_df.index.normalize().unique()
print(f"🚀 Starting backtest for {len(unique_days)} unique days...")

intial_capital = 100000
position_cap = 100000

# Track local trade log
local_trade_log2 = pd.DataFrame(columns=['name', 'Side', 'entry_date', 'entry_price' , 'qty' ,  'exit_date', 'exit_price', 'order_no','profit' ])

# ✅ Initialize a list to hold all trade dictionaries.
final_trade_log_list = []


# --- Main Loop: Iterate Through Each Day ---
for day in unique_days[2:4]:
    
    day_str = day.strftime('%Y-%m-%d')
    print(f"\n--- Processing Day: {day_str} ---")

    try:
        # Get the underlying data for today
        daily_underlying_df = com_df[com_df.index.date == day.date()]
        if daily_underlying_df.empty: continue
        
        print(f"Underlying data is {daily_underlying_df}")

        open_price = daily_underlying_df['SPY_close'].iloc[0]
        print(f"Fetching options across {open_price}")
        
        if pd.isna(open_price): continue
        
        atm_strike_open = np.round(open_price)

        # ==============================================================================
        # --- PHASE 1: PRE-FETCH ALL POTENTIAL OPTION DATA FOR THE DAY ---
        # ==============================================================================
        
        strikes_to_fetch = set()
        expiries_to_fetch = set()
        legs_to_trade_info = {} # Store info to map data back to legs

        for leg_name, params in strategy_config.items():
            center_target_strike = atm_strike_open 
                        # 1. Calculate the lower and upper price bounds
            lower_bound = center_target_strike * 0.99
            upper_bound = center_target_strike * 1.01

            # 2. Round the bounds to the nearest valid strike
            rounded_lower = round_to_nearest(lower_bound, interval=strike_interval)
            rounded_upper = round_to_nearest(upper_bound, interval=strike_interval)

            # 3. Generate all strikes from the lower to upper bound, inclusive
            all_strikes_in_range = np.arange(rounded_lower, rounded_upper + strike_interval, strike_interval)

                        
            # 4. Add all of these strikes to your set for fetching
            strikes_to_fetch.update(all_strikes_in_range)

            # print(f"\nDEBUG: Final unique strikes to be fetched: {sorted(list(strikes_to_fetch))}")

            schedule = nyse.schedule(start_date=day, end_date=day + pd.Timedelta(days=14))
            target_expiry_str = schedule.index[params['expiry_offset']].strftime('%Y-%m-%d')
            expiries_to_fetch.add(target_expiry_str)
            
            # Store the central parameters for this leg
            legs_to_trade_info[leg_name] = {'strike': center_target_strike, 'expiry': target_expiry_str}

        # Make one single, efficient call to fetch all data
        bulk_data_df = fetch_bulk_option_data(
            con=con, ticker='SPY', end_date_str=day_str, n_days=lookback_days,
            strikes=list(strikes_to_fetch), expiries=list(expiries_to_fetch), option_type=-1
        )

        if bulk_data_df.empty:
            print("Could not pre-fetch any option data for the day. Skipping.")
            continue
        grps = bulk_data_df.groupby(by=['strike' , 'optionType' , 'expiry'])
        total_legs =grps.ngroups 
        print(f"Total options found for today are: {total_legs}")       


                # Assume 'grps' is your DataFrameGroupBy object and 'daily_underlying_df' has SPY data for the day

        # ==============================================================================
        # --- PHASE 2: PREPARE THE MASTER SIMULATION DATAFRAME ---
        # ==============================================================================

        print("\n--- Phase 2: Preparing Master DataFrame ---")

        # 1. Create a list to hold the processed DataFrame for each option contract
        all_option_dfs = []

        # 2. Loop through each group (each unique option contract)
        for (strike, option_type, expiry), contract_df in grps:
            
            # 3. Create a unique, descriptive prefix for this specific contract
            # Example: 'C_468.0_20240105'
            type_char = 'C' if option_type == 1 else 'P'
            expiry_str = pd.to_datetime(expiry).strftime('%Y%m%d')
            prefix = f"{type_char}_{strike}_{expiry_str}_"
            
            # 4. Add the prefix to all columns of this contract's DataFrame
            prefixed_df = contract_df.add_prefix(prefix)
            
            # 5. Add the prepared DataFrame to our list
            all_option_dfs.append(prefixed_df)

        # 6. Concatenate all individual option DataFrames into one wide DataFrame
        if all_option_dfs:
            com_legs_df = pd.concat(all_option_dfs, axis=1, join='outer')
                       
            
            print(f"✅ Successfully created master DataFrame with {len(com_legs_df.columns)} total columns.")
            
        else:
            print("❌ No valid option groups to process.")
            

         
        legs_to_process  = [ symbol for symbol in com_legs_df.filter(like='_close').columns.str.replace('_close', '').unique()
                                if symbol.startswith('C_') or symbol.startswith('P_')]

        #print(legs_to_process)

        com_legs_df[[f"{legs}_intraday_low" for legs in legs_to_process]] = com_legs_df[[f"{legs}_bidPrice" for legs in legs_to_process]].apply(lambda x: intraday_low(com_legs_df, low_col=x.name,agg_func='min'))

        com_legs_df[[f"{legs}_intraday_high" for legs in legs_to_process]] = com_legs_df[[f"{legs}_askPrice" for legs in legs_to_process]].apply(lambda x: intraday_high(com_legs_df, high_col=x.name,agg_func='max'))

        close_cols = [f'{leg}_close' for leg in legs_to_process]
        com_legs_df[[f'{leg}_ma10' for leg in legs_to_process]] =  com_legs_df[close_cols].rolling(window=5).mean()

        # Calculate Today's Opening Price for each leg
        com_legs_df[[f'{leg}_todayo' for leg in legs_to_process]] =  com_legs_df[[f'{leg}_close' for leg in legs_to_process]].resample('D').transform('first')

        # Calculate Previous Day's Low (using bidPrice) for each leg
        com_legs_df[[f"{leg}_prevdayl" for leg in legs_to_process]] = com_legs_df[[f"{leg}_bidPrice" for leg in legs_to_process]].apply(lambda x: get_x_day_low_numba(com_legs_df, n=1, column=x.name))

        # Calculate Previous Day's High (using askPrice) for each leg
        com_legs_df[[f"{leg}_prevdayh" for leg in legs_to_process]] = \
            com_legs_df[[f"{leg}_askPrice" for leg in legs_to_process]].apply(lambda x: get_x_day_high_numba(com_legs_df, n=1, column=x.name) )

        
        window_size = 15

        # # 1. Rolling 15-Bar Highest Close
        # com_legs_df[[f'{leg}_15min_high' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).max()

        # # 2. Rolling 15-Bar Lowest Close
        # com_legs_df[[f'{leg}_15min_low' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).min()

        # # 3. Rolling 15-Bar Mean Close
        # com_legs_df[[f'{leg}_15min_mean' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).mean()

        com_legs_df[[f'{leg}_ROC_5' for leg in legs_to_process]] = com_legs_df[[f"{leg}_close" for leg in legs_to_process]].pct_change(periods=5)

        
        # 7. Join the wide option data with the underlying SPY data
        #com_df3 = pd.concat([daily_underlying_df, com_legs_df], join='outer', axis=1)
        
        com_df3 = com_legs_df.join(daily_underlying_df)
               
        # 1. Calculate the rounded ATM strike for every row
        atm_strikes = round_to_nearest(com_df3['SPY_close'], interval=1)
        atm1_strikes = round_to_nearest(com_df3['SPY_close'], interval=1) + 1

        # 2. Build the corresponding leg name for every row
        # This creates a pandas Series where each value is a string like 'C_468.0_20240105'
        com_df3['current_atm_leg'] = atm_strikes


            # 2. Initialize state dictionaries, now including position_side
        is_in_position = {}
        entry_price = {}
        entry_timestamp = {} # <-- ADD THIS LINE
        unrealized_pnl = {}
        position_side = {} # <-- ADD THIS
        bar_count = {}
        qty = {}
        entry_strike = {}
                
            # The main loop now iterates using an integer index 'i'
        for i in range(len(com_df3)):
            
            # Get the timestamp and row data using the integer index 'i'
            timestamp = com_df3.index[i]
            row = com_df3.iloc[i]
            
            # --- Part A: Update PnL & Check Exits for OPEN positions ---
            for open_leg_name in list(is_in_position.keys()):
                #print(f"Found position in {open_leg_name}")

                side = position_side[open_leg_name]
                current_price = row[f'{open_leg_name}_close']
                
                pnl = (current_price - entry_price[open_leg_name]) * qty[open_leg_name] * side
                unrealized_pnl[open_leg_name] = pnl
                bar_count[open_leg_name] += 1

                # --- Part B: Check for NEW entries ---
            for leg_name in legs_to_process:
                #print(f"Leg name is {leg_name}")
                if not is_in_position.get(leg_name):
                # and row['SPY_close']>row['SPY_intraday_high']*0.997                                         
                                        # Add the 'f' prefix to all strings accessing columns with leg_name
                                        # and 
                        #row[f'{leg_name}_close'] < min((com_df3[f"{leg_name}_intraday_high"].iloc[i - 1] * 0.9), com_df3[f"{leg_name}_close"].iloc[i - 1] * 1.1)
                    entry_condition_met = (
                        time(9, 45) <= timestamp.time() <= time(13, 15) and  row['SPY_close']<row['SPY_intraday_high']*0.998 and
                        row[f"{leg_name}_close"] < 0.8 and row[f"{leg_name}_optionType"] ==-1 and  row[f'{leg_name}_close'] >(com_df3[f"{leg_name}_intraday_low"].iloc[i - 1] * 1.25) and 
                        row[f"{leg_name}_strike"] <= row['SPY_close']-2   and  row[f"{leg_name}_strike"] >= row['SPY_close']-3 and
                        row["SPY_3d_Pctl"] < 0.7 and row[f'{leg_name}_close'] < min((com_df3[f"{leg_name}_intraday_high"].iloc[i - 1] * 0.9), com_df3[f"{leg_name}_close"].iloc[i - 1] * 1.25) and 
                        row[f"{leg_name}_ROC_5"] > 0.2 and row[f'{leg_name}_close']>row[f'{leg_name}_ma10']+0.05  )

                    short_condition_met = (
                        time(15, 45) <= timestamp.time() <= time(13, 45) and row['SPY_intraday_low']>row['SPY_todayo']*0.999 and
                        row[f"{leg_name}_close"] > 0.15 and row[f"{leg_name}_optionType"] ==-1 and
                        row[f"{leg_name}_intraday_high"] <1 and row[f"{leg_name}_strike"] >= row['SPY_close']-4 and row[f"{leg_name}_strike"] < row['SPY_close']-3 and 
                        row[f'{leg_name}_close'] < min((com_df3[f"{leg_name}_intraday_high"].iloc[i - 1] * 0.75), com_df3[f"{leg_name}_close"].iloc[i - 1] * 0.998) and
                        row["SPY_3d_Pctl"] > 0.7 and # Also better to access from 'row' for current timestamp data
                        row[f"{leg_name}_ROC_5"] < -0.01  )



                    # --- Check for a BUY Entry ---
                    if entry_condition_met:
                        
                        is_in_position[leg_name] = True
                        position_side[leg_name] = 1 # <-- Set side to 1 for long
                        entry_price[leg_name] = row[f'{leg_name}_askPrice']
                        qty[leg_name] = 1000
                        bar_count[leg_name] = 0
                        entry_timestamp[leg_name] = timestamp
                        entry_strike[leg_name] = row[f'{leg_name}_strike']
                        print(f"  -> BUY ENTRY for {leg_name} at {timestamp.time()} and price {entry_price[leg_name]}")

                    if short_condition_met:
                        
                        is_in_position[leg_name] = True
                        position_side[leg_name] = -1 # <-- Set side to 1 for long
                        entry_price[leg_name] = row[f'{leg_name}_askPrice']
                        qty[leg_name] = 750
                        bar_count[leg_name] = 0
                        entry_timestamp[leg_name] = timestamp
                        entry_strike[leg_name] = row[f'{leg_name}_strike']
                        print(f"  -> SELL ENTRY for {leg_name} at {timestamp.time()} and price {entry_price[leg_name]}")


                if is_in_position.get(leg_name):


                    sell_condition_met = timestamp.time() >= time(14, 45) and row[f'{leg_name}_bidPrice']<com_df3[f'{leg_name}_bidPrice'].iloc[i-1]-0.01
                    #sell_condition_met3 = timestamp.time() >= time(15, 45) 

                    sell_condition_2 = row[f'{leg_name}_bidPrice']>entry_price[leg_name]+0.45 and row[f'{leg_name}_bidPrice']>com_df3[f'{leg_name}_bidPrice'].iloc[i-1]*1.01

                    sell_condition_3 = row[f'{leg_name}_bidPrice']<entry_price[leg_name]-0.25 and row[f'{leg_name}_bidPrice']<com_df3[f'{leg_name}_bidPrice'].iloc[i-1]*0.99

                    sell_condition_4 = row[f'{leg_name}_close']<com_df3[f'{leg_name}_intraday_low'].iloc[i-1]*1.05 

                    if sell_condition_met :
                        print(f"EOD CLosing of {position_side[leg_name]} side positon in {leg_name} at {timestamp}")    
                            # 1. Get the Exit Price (use bid price since you are selling to close)
                        exit_price = row[f'{leg_name}_bidPrice']
                        side = position_side[leg_name]
                            # ... etc.
                            # 2. Calculate the final profit
                        profit = (exit_price - entry_price[leg_name]) * qty[leg_name] * side * option_multiplier

                        # 3. Create a dictionary for the trade log
                        trade_details = {
                            'name': leg_name,
                            'Side': 'BUY' if side == 1 else 'SELL',
                            'entry_date': entry_timestamp[leg_name], # We will add this in the next step
                            'entry_price': entry_price[leg_name],
                            'qty': qty[leg_name],
                            'exit_date': timestamp,
                            'exit_price': exit_price,
                            'profit': profit,
                            'bars_held': bar_count[leg_name]
                        }
                        
                        # 4. Append the log to our final list
                        final_trade_log_list.append(trade_details)
                        
                        print(f"  -> EOD EXIT for {leg_name} at {exit_price:.2f}. Profit: ${profit:.2f}")

                        # 5. Reset all state dictionaries for this leg
                        del is_in_position[leg_name]
                        del position_side[leg_name]
                        del entry_price[leg_name]
                        del qty[leg_name]
                        del bar_count[leg_name]
                        del unrealized_pnl[leg_name]
                        del entry_timestamp[leg_name] # Also remove the entry timestamp
                    
                    if sell_condition_3 :
                        
                            
                        exit_price = row[f'{leg_name}_bidPrice']
                        side = position_side[leg_name]
                            # ... etc.
                            # 2. Calculate the final profit
                        profit = (exit_price - entry_price[leg_name]) * qty[leg_name] * side * option_multiplier
                        print(f"Stoploss of  {position_side[leg_name]} side entry was {entry_price[leg_name]} in {leg_name} at {exit_price}")    
                        # 3. Create a dictionary for the trade log
                        trade_details = {
                            'name': leg_name,
                            'Side': 'BUY' if side == 1 else 'SELL',
                            'entry_date': entry_timestamp[leg_name], # We will add this in the next step
                            'entry_price': entry_price[leg_name],
                            'qty': qty[leg_name],
                            'exit_date': timestamp,
                            'exit_price': exit_price,
                            'profit': profit,
                            'bars_held': bar_count[leg_name]
                        }
                        
                        # 4. Append the log to our final list
                        final_trade_log_list.append(trade_details)
                        
                        print(f"  -> EOD EXIT for {leg_name} at {exit_price:.2f}. Profit: ${profit:.2f}")

                        # 5. Reset all state dictionaries for this leg
                        del is_in_position[leg_name]
                        del position_side[leg_name]
                        del entry_price[leg_name]
                        del qty[leg_name]
                        del bar_count[leg_name]
                        del unrealized_pnl[leg_name]
                        del entry_timestamp[leg_name] # Also remove the entry timestamp
                    


                    if sell_condition_2:
                       
                       
                        exit_price = row[f'{leg_name}_bidPrice']
                        side = position_side[leg_name]
                            # ... etc.
                            # 2. Calculate the final profit
                        profit = (exit_price - entry_price[leg_name]) * qty[leg_name] * side * option_multiplier

                        # 3. Create a dictionary for the trade log
                        trade_details = {
                            'name': leg_name,
                            'Side': 'BUY' if side == 1 else 'SELL',
                            'entry_date': entry_timestamp[leg_name], # We will add this in the next step
                            'entry_price': entry_price[leg_name],
                            'qty': qty[leg_name],
                            'exit_date': timestamp,
                            'exit_price': exit_price,
                            'profit': profit,
                            'bars_held': bar_count[leg_name]
                        }
                        
                        # 4. Append the log to our final list
                        final_trade_log_list.append(trade_details)
                        
                        print(f"  -> Target EXIT for {leg_name} at {exit_price:.2f}. Profit: ${profit:.2f}")

                        # 5. Reset all state dictionaries for this leg
                        del is_in_position[leg_name]
                        del position_side[leg_name]
                        del entry_price[leg_name]
                        del qty[leg_name]
                        del bar_count[leg_name]
                        del unrealized_pnl[leg_name]
                        del entry_timestamp[leg_name] # Also remove the entry timestamp

                    if sell_condition_4:
                       
                       
                        exit_price = row[f'{leg_name}_bidPrice']
                        side = position_side[leg_name]
                            # ... etc.
                            # 2. Calculate the final profit
                        profit = (exit_price - entry_price[leg_name]) * qty[leg_name] * side * option_multiplier

                        # 3. Create a dictionary for the trade log
                        trade_details = {
                            'name': leg_name,
                            'Side': 'BUY' if side == 1 else 'SELL',
                            'entry_date': entry_timestamp[leg_name], # We will add this in the next step
                            'entry_price': entry_price[leg_name],
                            'qty': qty[leg_name],
                            'exit_date': timestamp,
                            'exit_price': exit_price,
                            'profit': profit,
                            'bars_held': bar_count[leg_name]
                        }
                        
                        # 4. Append the log to our final list
                        final_trade_log_list.append(trade_details)
                        
                        print(f"  -> MA cross down exit of {leg_name} at {exit_price:.2f}. Profit: ${profit:.2f}")

                        # 5. Reset all state dictionaries for this leg
                        del is_in_position[leg_name]
                        del position_side[leg_name]
                        del entry_price[leg_name]
                        del qty[leg_name]
                        del bar_count[leg_name]
                        del unrealized_pnl[leg_name]
                        del entry_timestamp[leg_name] # Also remove the entry timestamp



                   


        
    except Exception as e:
        print(f"  ❌ An error occurred on {day_str}: {e}")





🚀 Starting backtest for 147 unique days...

--- Processing Day: 2024-01-05 ---
Underlying data is                        SPY_close    VIX_open    VIX_high     VIX_low  \
option_ts                                                              
2024-01-05 09:36:00 468.15000000 13.59000000 13.60000000 13.59000000   
2024-01-05 09:38:00 468.14000000 13.58000000 13.59000000 13.58000000   
2024-01-05 09:39:00 468.20000000 13.57000000 13.61000000 13.56000000   
2024-01-05 09:40:00 467.94000000 13.63000000 13.67000000 13.63000000   
2024-01-05 09:41:00 467.67000000 13.65000000 13.68000000 13.65000000   
...                          ...         ...         ...         ...   
2024-01-05 13:22:00 467.01000000 13.68000000 13.68000000 13.66000000   
2024-01-05 13:24:00 466.97000000 13.68000000 13.68000000 13.66000000   
2024-01-05 13:25:00 467.12000000 13.65000000 13.65000000 13.60000000   
2024-01-05 13:26:00 467.37000000 13.59000000 13.59000000 13.57000000   
2024-01-05 13:28:00 467.54000000 13.57

In [ ]:
com_df3[['SPY_close' , 'P_463.0_20240108_close' , 'P_463.0_20240108_' ]]

,SPY_close,P_463.0_20240108_close
ts,,
2024-01-08 09:30:00,468.43000000,0.00000000
2024-01-08 09:31:00,NaN,0.03500000
2024-01-08 09:32:00,NaN,0.02500000
2024-01-08 09:33:00,NaN,0.02500000
2024-01-08 09:34:00,NaN,0.02500000
...,...,...
2024-01-08 15:56:00,NaN,0.00500000
2024-01-08 15:57:00,NaN,0.00500000
2024-01-08 15:58:00,NaN,0.00500000


In [ ]:
import numpy as np
from scipy.stats import norm

def calculate_delta_vectorized(S, K, T_years, r, IV, option_type_series):
    """
    Calculates Delta for all rows of data at once using NumPy.
    
    Args:
        S (pd.Series): Underlying prices
        K (pd.Series): Strike prices
        T_years (pd.Series): Time to expiry in years
        r (float): Risk-free interest rate
        IV (pd.Series): Implied Volatility
        option_type_series (pd.Series): Column with 1 for calls, -1 for puts
        
    Returns:
        np.ndarray: An array of delta values.
    """
    # Add a small epsilon to prevent division by zero in edge cases
    IV = IV + 1e-9 
    T_years = T_years + 1e-9
    
    d1 = (np.log(S / K) + (r + 0.5 * IV**2) * T_years) / (IV * np.sqrt(T_years))
    
    # Calculate delta using the CDF and np.where for conditional logic
    call_delta = norm.cdf(d1)
    put_delta = call_delta - 1
    
    # Use np.where to apply the correct formula based on option type
    delta = np.where(option_type_series == 1, call_delta, put_delta)
    
    return delta

In [ ]:
for leg_name in legs_to_process:
    # Check if all necessary columns exist for this leg
    required_cols = [
        f'{leg_name}_strike', f'{leg_name}_dte', 
        f'{leg_name}_iv', f'{leg_name}_optionType', 'SPY_close'
    ]

    print(required_cols)

    break

In [ ]:
com_legs_df[]

In [ ]:
com_legs_df.filter(regex='P_463').columns

In [ ]:
com_df3

In [ ]:
# (This line should already be in your code)
# com_df3 = pd.concat([daily_underlying_df, com_legs_df], join='outer', axis=1)

print("\n--- Calculating Delta for all option legs ---")

# Define the risk-free rate
risk_free_rate = 0.03

# Loop through each unique option leg you've processed
for leg_name in legs_to_process:
    # Check if all necessary columns exist for this leg in the CORRECT DataFrame
    required_cols = [
        f'{leg_name}_strike', f'{leg_name}_dte',
        f'{leg_name}_iv', f'{leg_name}_optionType', 'SPY_close'
    ]
    # ▼▼▼ FIX: Check in com_df3 ▼▼▼
    if not all(col in com_df3.columns for col in required_cols):
        print(f"  -> Skipping {leg_name}, missing required data in com_df3.")
        continue

    # Create the new delta column for the current leg in the CORRECT DataFrame
    # ▼▼▼ FIX: Use com_df3 for all inputs and to store the result ▼▼▼
    com_df3[f'{leg_name}_delta'] = calculate_delta_vectorized(
        S=com_df3['SPY_close'],
        K=com_df3[f'{leg_name}_strike'],
        T_years=com_df3[f'{leg_name}_dte'] / 365.0,
        r=risk_free_rate,
        IV=com_df3[f'{leg_name}_iv'],
        option_type_series=com_df3[f'{leg_name}_optionType']
    )

print("✅ Delta calculations complete.")

# Display the first few rows of some new delta columns to verify
delta_cols_to_show = [col for col in com_df3.columns if '_delta' in col]
print("\nSample of calculated Delta columns:")
print(com_df3[delta_cols_to_show].head())

In [ ]:
delta_cols_to_show

In [ ]:
local_trade_log2 = pd.DataFrame(final_trade_log_list)

In [ ]:
local_trade_log2

In [ ]:
daily_profits = local_trade_log2.groupby(local_trade_log2['entry_date'].dt.date)['profit'].sum()

In [ ]:
com_legs_df

In [ ]:
daily_profits.cumsum().plot()

In [ ]:
# ==============================================================================
# ---  HELPER FUNCTION for Clean and Fast Trade Logging ---
# ==============================================================================
def log_and_close_trade(trade_log_list, leg, exit_timestamp, exit_price, exit_reason, state_vars):
    """
    Calculates P&L, appends the trade to a log list, and resets the state for that leg.
    """
    # Unpack state variables for easier access
    entry_dates, entry_prices, qty, position_type, instrument_positions, bar_count = (
        state_vars['entry_dates'], state_vars['entry_prices'], state_vars['qty'],
        state_vars['position_type'], state_vars['instrument_positions'], state_vars['bar_count']
    )
    
    if entry_prices[leg] is None: return # Avoid logging if no entry price exists

    if position_type[leg] == 1:  # Long position
        profit = (exit_price - entry_prices[leg]) * qty[leg]
        side = 'BUY'
    else: # Short position
        profit = (entry_prices[leg] - exit_price) * qty[leg]
        side = 'SELL'

    trade_log_list.append({
        'name': leg, 'Side': side, 'entry_date': entry_dates[leg],
        'entry_price': entry_prices[leg], 'qty': qty[leg],
        'exit_date': exit_timestamp, 'exit_price': exit_price,
        'profit': profit, 'exit_reason': exit_reason
    })

    # Reset state variables
    instrument_positions[leg] = False
    entry_prices[leg] = None
    bar_count[leg] = 0
    
    print(f"   -> EXIT {leg} ({side}) at {exit_price:.2f} for {exit_reason}. P&L: ${profit:.2f}")

# ==============================================================================

In [ ]:
if 'entry_date' in local_trade_log2.columns:

    local_trade_log2.set_index('entry_date' , inplace=True)

    local_trade_log2.sort_index( ascending=True, inplace=True)

if 'exit_date' in local_trade_log2.columns:
    local_trade_log2['exit_date'] = pd.to_datetime(local_trade_log2['exit_date'])

# Calculate the absolute percentage difference between entry and exit prices
local_trade_log2['price_change_pct'] = (local_trade_log2['exit_price'] - local_trade_log2['entry_price']).abs() / local_trade_log2['entry_price']

# Count trades before filtering
trades_before_filtering = len(local_trade_log2)

average_trade_profit = local_trade_log2['profit'].mean()

# First ensure consistent case for Side column

if 'side' in local_trade_log2.columns:
    local_trade_log2.rename(columns={'side': 'Side'}, inplace=True)


local_trade_log2['Side'] = local_trade_log2['Side'].str.lower()

# Calculate profit in bps (handles both buy/long and sell/short)
local_trade_log2['profit_bps'] = np.where(
    local_trade_log2['Side'].isin(['buy', 'long']),
    ((local_trade_log2['exit_price'] - local_trade_log2['entry_price']) / local_trade_log2['entry_price']) * 10000,
    ((local_trade_log2['entry_price'] - local_trade_log2['exit_price']) / local_trade_log2['entry_price']) * 10000)

# open_positions = list(long_position_count.values())
# print(f"Max open long positions are: {np.max(open_positions)}")

# open__short_positions = list(short_position_count.values())
# print(f"Max open shrt positions are: {np.max(open__short_positions)}")

total_trade_count = len(local_trade_log2)
max_entries_per_day = local_trade_log2.groupby(local_trade_log2.index.date).size().max()

# Identify losing trades and percentage

losers = local_trade_log2[local_trade_log2['profit'] < 0]
num_losing_trades = len(losers)
losers_percentage = (num_losing_trades / total_trade_count) * 100 if total_trade_count > 0 else 0

winners = local_trade_log2[local_trade_log2['profit'] > 0]

# # 1. Create a pandas Series from your PnL data
# pnl_series = pd.Series(total_pnl_series)

# # 2. Find the minimum value in the series
# max_unrealized_loss = pnl_series.min()

# print(f"The highest unrealized loss (minimum PnL) was: ${max_unrealized_loss:,.2f}")

# Updated side checks throughout
num_long_trades = len(local_trade_log2[local_trade_log2['Side'].isin(['buy', 'long'])])
num_short_trades = len(local_trade_log2[local_trade_log2['Side'].isin(['sell', 'short'])])
print(f"Number of long trades: {num_long_trades} and short trades: {num_short_trades} out of total trades: {total_trade_count}")

# Updated profit calculations
avg_long_profit_bps = local_trade_log2[local_trade_log2['Side'].isin(['buy', 'long'])]['profit_bps'].mean()
avg_short_profit_bps = local_trade_log2[local_trade_log2['Side'].isin(['sell', 'short'])]['profit_bps'].mean()
print(f"Average long trade profit: {avg_long_profit_bps:.2f} bps and short: {avg_short_profit_bps:.2f} bps")

profit_total_long = local_trade_log2[local_trade_log2['Side'].isin(['buy', 'long'])]['profit'].sum()
profit_total_short = local_trade_log2[local_trade_log2['Side'].isin(['sell', 'short'])]['profit'].sum()
print(f"Net long profit: {profit_total_long:.2f} and short profit: {profit_total_short:.2f}")

# Compute holding period
local_trade_log2['holding_days'] = (local_trade_log2['exit_date'] - local_trade_log2.index).dt.days

# Compute equity curve (EC)
local_trade_log2['EC'] = (position_cap*100) + local_trade_log2['profit'].cumsum()

#============================================

# The index should be the 'exit_date' because that's when the equity changes
equity_at_exit = pd.Series(data=local_trade_log2['EC'].values, index=pd.to_datetime(local_trade_log2['exit_date']))

print(f"Equity at exit is : {equity_at_exit}")

# Print average trade profit
print(f"Average Trade Profit: {average_trade_profit:.2f}")

# Print profit in basis points (bps)
print("Profit (bps) for each trade:")
print(f"Avergae trade profit in bps : {local_trade_log2['profit_bps'].mean()}" )

# Print total trade count
print(f"Total Trade Count: {total_trade_count}")

# Print max entries per day
print(f"Max Entries Per Day: {max_entries_per_day}")

# Print number of losing trades
print(f"Number of Losing Trades: {num_losing_trades}")

print(f"Losing Trades Percentage: {losers_percentage:.2f}%")

# Print holding period
print("Avg Holding Period in (Days) :")
print(local_trade_log2['holding_days'].mean())

# Add: Number of trades with loss greater than 10% (i.e. -1000 bps)
loss_threshold_bps = -1000  # -10%
deep_losers = local_trade_log2[local_trade_log2['profit_bps'] < loss_threshold_bps]
num_deep_losers = len(deep_losers)

print(f"Number of Losing Trades > 10% Loss: {num_deep_losers}")

# Add: Average loss of all losing trades
avg_losing_trade = losers['profit'].mean()
print(f"Average Loss per Losing Trade: {avg_losing_trade:.4f}")

# Calculate average profit percentage (in % terms)
average_profit_percent = local_trade_log2['profit'].mean() / position_cap * 100
print(f"\nAverage Profit Percentage per Trade: {average_profit_percent:.2f}%")

# Calculate Trade Expectancy (in monetary terms)
win_rate = len(winners) / total_trade_count
avg_win = winners['profit'].mean()
avg_loss = losers['profit'].mean()
trade_expectancy = (win_rate * avg_win) - ((1 - win_rate) * abs(avg_loss))
print(f"Trade Expectancy: ${trade_expectancy:.2f} per trade")

# Calculate Risk-Adjusted Metrics
risk_reward_ratio = abs(avg_win / avg_loss) if avg_loss != 0 else float('inf')
print(f"Risk-Reward Ratio: {risk_reward_ratio:.2f}:1")


# Add: Average loss of all losing trades
avg_winning_trade = winners['profit'].mean()
print(f"Average Loss per Losing Trade: {avg_winning_trade:.4f}")


abs_profit_df =  pd.DataFrame()
for symbol , data in absolute_symbol_pnl_series.items():

    tdf = pd.DataFrame(data , columns=['timestamp' , symbol])
    tdf.set_index('timestamp' , inplace=True)

    # Reindex to match df_index, filling missing values with 0
    abs_profit_df[symbol] = tdf.reindex(com_df2.index, fill_value=0)

max_unrealised_drawdown = abs_profit_df.sum(axis=1).min()
print(f"Max unrealised loss ( which is unrealised drawdown on notional) is {max_unrealised_drawdown}")


# Long/Short specific metrics
# Update all calculations to use the new condition
long_trades = local_trade_log2[local_trade_log2['Side'].isin(['buy', 'long'])]
short_trades = local_trade_log2[local_trade_log2['Side'].isin(['sell', 'short'])]

num_long_trades = len(long_trades)
num_short_trades = len(short_trades)

# Profit metrics
avg_long_profit_bps = long_trades['profit_bps'].mean()
avg_short_profit_bps = short_trades['profit_bps'].mean()
profit_total_long = long_trades['profit'].sum()
profit_total_short = short_trades['profit'].sum()

# Win/loss metrics
def calculate_consecutive_wins_losses(trades):
    trades = trades.sort_index()
    consecutive = []
    current_streak = 0
    prev_result = None

    for profit in trades['profit']:
        current_result = 'win' if profit > 0 else 'loss'
        if current_result == prev_result or prev_result is None:
            current_streak += 1
        else:
            consecutive.append((prev_result, current_streak))
            current_streak = 1
        prev_result = current_result

    if current_streak > 0:
        consecutive.append((prev_result, current_streak))

    return pd.DataFrame(consecutive, columns=['type', 'length'])

# Calculate for long and short separately
long_consecutive = calculate_consecutive_wins_losses(long_trades)
short_consecutive = calculate_consecutive_wins_losses(short_trades)

def print_streak_stats(name, df):
    if len(df) > 0:
        max_win_streak = df[df['type'] == 'win']['length'].max()
        max_loss_streak = df[df['type'] == 'loss']['length'].max()
        avg_win_streak = df[df['type'] == 'win']['length'].mean()
        avg_loss_streak = df[df['type'] == 'loss']['length'].mean()
        print(f"\n{name} Streaks:")
        print(f"Max Winning Streak: {max_win_streak}")
        print(f"Max Losing Streak: {max_loss_streak}")
        print(f"Avg Winning Streak: {avg_win_streak:.1f}")
        print(f"Avg Losing Streak: {avg_loss_streak:.1f}")

print_streak_stats("Long", long_consecutive)
print_streak_stats("Short", short_consecutive)

# Winning ratios
long_winners = len(long_trades[long_trades['profit'] > 0])
short_winners = len(short_trades[short_trades['profit'] > 0])

long_win_ratio = long_winners / num_long_trades if num_long_trades > 0 else 0
short_win_ratio = short_winners / num_short_trades if num_short_trades > 0 else 0

print(f"\nLong Trades Win Ratio: {long_win_ratio:.2%}")
print(f"\nShort Trades Win Ratio: {short_win_ratio:.2%}")

# Print all metrics
print("\n==========================================")
print(f"Number of long trades: {num_long_trades}")
print(f"Number of short trades: {num_short_trades}")
print(f"Total trades: {total_trade_count}")

print(f"\nAverage long trade profit: {avg_long_profit_bps:.2f} bps")
print(f"Average short trade profit: {avg_short_profit_bps:.2f} bps")
print(f"Net long profit: {profit_total_long:.2f}")
print(f"Net short profit: {profit_total_short:.2f}")

import matplotlib.pyplot as plt

# Scatter plot of trades
plt.figure(figsize=(12, 6))

# X-axis: trade index (or entry date if you want timeline)
x = range(len(local_trade_log2))

# Y-axis: profit
y = local_trade_log2['profit']

# Scatter: green for winners, red for losers
colors = ['green' if p > 0 else 'red' for p in y]

plt.scatter(x, y, c=colors, alpha=0.7, edgecolors='k')

# Add horizontal line at 0 (break-even)
plt.axhline(0, color='black', linestyle='--', linewidth=1)

plt.title("Scatter of Trade Profits")
plt.xlabel("Trade Index")
plt.ylabel("Profit ($)")
plt.show()



from IPython.display import HTML
import matplotlib.pyplot as plt
import base64
from io import BytesIO

# Create equity curve plot
plt.figure(figsize=(10, 6))
local_trade_log2['EC'].plot(title='Equity Curve', grid=True)
plt.ylabel('Portfolio Value')
plt.tight_layout()

# Save plot to temporary buffer
buffer = BytesIO()
plt.savefig(buffer, format='png')
plt.close()
plot_data = base64.b64encode(buffer.getvalue()).decode('utf-8')


#=== symbols specific

# 2. Group by the symbol name and calculate the sum of profits for each.
symbol_profits = local_trade_log2.groupby('name')['profit'].sum()

# Display the total profit for each symbol
print("--- Total Profit per Symbol ---")
print(symbol_profits)


# 3. Sort the results to easily find the best and worst.
sorted_symbols = symbol_profits.sort_values(ascending=False)


# 4. Extract the best and worst symbols from the sorted list.
# The best symbol is the first item (index 0).
best_symbol = sorted_symbols.index[0]
best_profit = sorted_symbols.iloc[0]

# The worst symbol is the last item (index -1).
worst_symbol = sorted_symbols.index[-1]
worst_loss = sorted_symbols.iloc[-1]


# 3. Sort the profits in ASCENDING order.
#    This places the symbols with the largest losses at the top.
sorted_by_loss = symbol_profits.sort_values(ascending=True)


# 4. Select the first 10 rows from the sorted list using .head(10).
worst_10_symbols = sorted_by_loss.head(10)


# 5. Print the final result.
print("--- 10 Worst Performing Symbols (by Total Profit) ---")
print(worst_10_symbols)

#======================================



# --- NEW: Calculate and print the additional statistics ---

# Count the number of symbols with negative total profit (losers)
num_loss_making_symbols = (symbol_profits <= 0).sum()

# Count the number of symbols with positive total profit (winners)
num_profit_making_symbols = (symbol_profits > 0).sum()

# Calculate the ratio of losers to winners
# Add a check to prevent division by zero if there are no winners
if num_profit_making_symbols > 0:
    loss_to_profit_ratio = num_loss_making_symbols / num_profit_making_symbols
else:
    loss_to_profit_ratio = float('inf') # Indicate infinite ratio if no winners

# Print the new stats
print("\n--- Symbol Profit/Loss Summary ---")
print(f"Number of Profit-Making Symbols: {num_profit_making_symbols}")
print(f"Number of Loss-Making Symbols:   {num_loss_making_symbols}")
print(f"Ratio of Loss-Makers to Profit-Makers: {loss_to_profit_ratio:.2f}")




def find_max_daily_gmv(trade_log):

    """
    Calculates the maximum and mean Gross Merchandise Value (GMV) on a single day
    from a trade log DataFrame. The mean calculation accounts for all days in the
    period, including non-trading days.

    Args:
        trade_log (pd.DataFrame): DataFrame containing the trade log. 
                                  Index must be a datetime object ('entry_date').
                                  Must contain 'entry_price', 'exit_price', 
                                  'qty', and 'exit_date' columns.

    Returns:
        tuple: A tuple containing the max_gmv_value and the mean_gmv_value.
    """

    # Calculate the total value of each entry and exit transaction
    trade_log['entry_value'] = trade_log['entry_price'] * trade_log['qty']
    trade_log['exit_value'] = trade_log['exit_price'] * trade_log['qty']

    # Group all entry values by their calendar day and sum them up
    daily_entries = trade_log.groupby(trade_log.index.date)['entry_value'].sum()

    # Group all exit values by their calendar day and sum them up
    # CORRECTED: Group exits by 'exit_date', not the index ('entry_date')
    daily_exits = trade_log.groupby(trade_log['exit_date'].dt.date)['exit_value'].sum()

    # Combine the daily entry and exit sums to get the total GMV for each day
    # This series only contains days where a trade occurred.
    daily_gmv = daily_entries.add(daily_exits, fill_value=0)

    # Find the maximum value. This can be done before reindexing for efficiency.
    if daily_gmv.empty:
        return 0, 0
    max_gmv_value = daily_gmv.max()

    # --- MODIFICATION TO ACCOUNT FOR NON-TRADING DAYS ---

    # 1. Determine the full date range of the backtest period.
    #    We use the earliest entry and latest exit to define the period.
    start_date = trade_log.index.min().date()
    end_date = trade_log['exit_date'].max().date()
    all_days_in_period = pd.date_range(start=start_date, end=end_date, freq='D')

    # 2. Reindex the 'daily_gmv' Series to include all days.
    #    This assigns a GMV of 0 to all non-trading days.
    daily_gmv = daily_gmv.reindex(all_days_in_period, fill_value=0)
    
    # --- END OF MODIFICATION ---

    # Now, this calculation correctly finds the mean over the entire period.
    mean_gmv_value = daily_gmv.mean()

    return max_gmv_value, mean_gmv_value


max_gmv , mean_gmv = find_max_daily_gmv(local_trade_log2)

print(f"Average daily GMV is {mean_gmv}")
print(f"Max daily gmv is {max_gmv}")


print(f"Return on average GMV is {local_trade_log2['profit'].sum()/mean_gmv}")
print(f"Return on average GMV is {local_trade_log2['profit'].sum()/max_gmv}")



# Create HTML report
html_report = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Backtest Performance Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        h1 {{ color: #2c3e50; border-bottom: 2px solid #3498db; }}
        h2 {{ color: #2980b9; }}
        .metric-container {{ display: flex; flex-wrap: wrap; gap: 20px; }}
        .metric-card {{
            background: #f8f9fa;
            border: 1px solid #dee2e6;
            border-radius: 5px;
            padding: 15px;
            flex: 1 1 300px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .metric-title {{ font-weight: bold; color: #2c3e50; }}
        .metric-value {{ font-size: 1.2em; color: #27ae60; }}
        .negative {{ color: #e74c3c; }}
        .plot-container {{ margin: 30px 0; text-align: center; }}
        table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
        th, td {{ padding: 10px; text-align: left; border-bottom: 1px solid #ddd; }}
        th {{ background-color: #3498db; color: white; }}
        tr:nth-child(even) {{ background-color: #f2f2f2; }}
    </style>
</head>
<body>
    <h1>Strategy Backtest Performance Report</h1>

    <div class="plot-container">
        <h2>Equity Curve</h2>
        <img src="data:image/png;base64,{plot_data}" alt="Equity Curve">
    </div>

    <h2>Key Performance Metrics</h2>
    <div class="metric-container">
        <div class="metric-card">
            <div class="metric-title">Total Trades</div>
            <div class="metric-value">{total_trade_count:,}</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Win Rate</div>
            <div class="metric-value">{100 - losers_percentage:.1f}%</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Average Trade Profit</div>
            <div class="metric-value">${average_trade_profit:,.2f}</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Trade Expectancy</div>
            <div class="metric-value">${trade_expectancy:,.2f}</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Risk-Reward Ratio</div>
            <div class="metric-value">{risk_reward_ratio:.2f}:1</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Max Drawdown</div>
            <div class="metric-value negative">${max_unrealised_drawdown:,.2f}</div>
        </div>
    </div>

    <h2>Long/Short Breakdown</h2>
    <table>
        <tr>
            <th>Metric</th>
            <th>Long Trades</th>
            <th>Short Trades</th>
        </tr>
        <tr>
            <td>Count</td>
            <td>{num_long_trades:,}</td>
            <td>{num_short_trades:,}</td>
        </tr>
        <tr>
            <td>Win Rate</td>
            <td>{long_win_ratio:.1%}</td>
            <td>{short_win_ratio:.1%}</td>
        </tr>
        <tr>
            <td>Avg Profit (bps)</td>
            <td>{avg_long_profit_bps:.1f}</td>
            <td>{avg_short_profit_bps:.1f}</td>
        </tr>
        <tr>
            <td>Total Profit</td>
            <td>${profit_total_long:,.2f}</td>
            <td>${profit_total_short:,.2f}</td>
        </tr>
        <tr>
            <td>Max Win Streak</td>
            <td>{long_consecutive[long_consecutive['type']=='win']['length'].max() if len(long_consecutive) > 0 else 0}</td>
            <td>{short_consecutive[short_consecutive['type']=='win']['length'].max() if len(short_consecutive) > 0 else 0}</td>
        </tr>
        <tr>
            <td>Max Loss Streak</td>
            <td>{long_consecutive[long_consecutive['type']=='loss']['length'].max() if len(long_consecutive) > 0 else 0}</td>
            <td>{short_consecutive[short_consecutive['type']=='loss']['length'].max() if len(short_consecutive) > 0 else 0}</td>
        </tr>
    </table>

    <h2>Position Sizing</h2>
    <div class="metric-container">
        
        <div class="metric-card">
            <div class="metric-title">Max Entries Per Day</div>
            <div class="metric-value">{max_entries_per_day}</div>
        </div>
    </div>

    <h2>Trade Duration</h2>
    <div class="metric-container">
        <div class="metric-card">
            <div class="metric-title">Avg Holding Days</div>
            <div class="metric-value">{local_trade_log2['holding_days'].mean():.1f}</div>
        </div>
    </div>
</body>
</html>
"""

# Display the report
HTML(html_report)

# To save the report to a file
with open('Nasdaq_Vol_stocks.html', 'w') as f:
    f.write(html_report)

local_trade_log2.to_csv("Nasdaq_TL.csv", index=True)

# Convert total_pnl_series dict (timestamp → pnl) into DataFrame
mtm_equity_df = pd.DataFrame(list(total_pnl_series.items()), columns=["timestamp", "MTM_PnL"])

# Sort by timestamp (important to keep the curve ordered)
mtm_equity_df.sort_values("timestamp", inplace=True)

# Add equity column: starting capital + cumulative PnL
mtm_equity_df["Equity"] = intial_capital + mtm_equity_df["MTM_PnL"].cumsum()

# Set timestamp as index for convenience
mtm_equity_df.set_index("timestamp", inplace=True)

# Define full path
mtm_equity_file = os.path.join(trades_strategy_folder, "MTM_equity.csv")

# Save to CSV
mtm_equity_df.to_csv(mtm_equity_file)

print(f"✅ MTM equity curve saved at: {mtm_equity_file}")


# for symbol, data in absolute_symbol_pnl_series.items():
#     if not data:
#         print(f"No data for {symbol}, skipping.")
#         continue
#     try:
#         tdf = pd.DataFrame(data, columns=['timestamp', 'pnl_absolute'])
#         tdf.set_index('timestamp', inplace=True)
#         file_path = os.path.join(df_folder, f"{symbol}_pnl.csv")
#         tdf.to_csv(file_path)
#         print(f"Successfully saved {file_path}")
#     except Exception as e:
#         print(f"Error processing {symbol}: {e}")


import quantstats as qs
qs.extend_pandas()

qs.reports.basic(local_trade_log2['EC'])

# --- 2. Plotting the 'EC' column ---
plt.figure(figsize=(12, 6)) # Optional: Adjust the figure size

local_trade_log2['EC'].plot(grid=True) # The main plotting command

# --- 3. Add labels and a title for clarity ---
plt.title('EC Over Time')
plt.xlabel('Date')
plt.ylabel('EC Value')

# --- 4. Display the plot ---
plt.show()


In [ ]:
local_trade_log2['profit']